# 1. Keras Start

- 1.4 Simple CNN
- 1.5 BatchNormalization & Dropout

## 1.4 Simple CNN
이번 실습은 classifier 역할을 하는 DNN 앞에, feature extractor 역할을 하는 Covolution layer를 및 Maxplling layer를 덧붙여

CNN 모델을 만들고 학습시켜 볼 것이다.

## (1) 데이터셋

In [1]:
(X_train, Y_train),(X_test, Y_test) = datasets.mnist.load_data()
print(X_train.shape, Y_train.shape)

NameError: name 'datasets' is not defined

MNIST 데이터는 load했을 때 channel이 없기 때문에 channel을 추가하여 3차원 이미지로 바꿔주어야 함(batch차원 제외)

Tensorflow base에서는 (batch, image row, image column, image channel)으로 이미지를 학습

Teano base에서는 (batch, image channel, image row, image column)으로 이미지를 학습

backend.image_data_format()로 channel의 위치를 확인하고 reshape


In [ ]:
from tensorflow.keras import backend
backend.image_data_format()

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

Y_train = utils.to_categorical(Y_train)
Y_test = utils.to_categorical(Y_test)

print(X_train.shape, Y_train.shape)

n_in = X_train.shape[1:]
n_out = Y_train.shape[-1]

## (2) 모델링

<사용되는 Layer>

`Conv2D` : 이미지에 필터의 파라미터를 convolution 연산하여 다음 layer로 전달

https://keras.io/layers/convolutional/#conv2d

`MaxPooling2D` : 필터에 겹치는 값들 중 가장 큰 값만 다음 layer로 전달

https://keras.io/layers/pooling/#maxpooling2d

`Flatten` : 다차원 tensor를 1차원 벡터로 변환

https://keras.io/layers/core/#flatten

In [ ]:
from tensorflow.keras.layers import Flatten, BatchNormalization, Dropout, ReLU
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

In [2]:
def DNN_seq(n_in, n_out):
    # Coding Time (5 min) layer : 784 → 128 → 32 → 10, acitvation : relu → relu → softmax
    model = Sequential()
    model.add(Dense(units = 128, input_shape = (n_in,), activation = 'relu'))    # Dense : FClayer
    model.add(Dense(units = 32, input_shape = (128,), activation = 'relu'))
    model.add(Dense(units = 10, input_shape = (32,), activation = 'softmax'))
    return model

# model = DNN_seq(n_in, n_out)
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model=CNN(n_in, n_out)
model.summary()

from tensorflow.keras.utils import plot_model
%matplotlib inline
plot_model(model, show_shapes=True)

NameError: name 'CNN' is not defined

## (3) 모델의 학습과정 설정

optimizer에 문자열 대신에 파라미터를 수정한 optimizer를 입력할 수 있음

https://keras.io/optimizers/#adam

In [ ]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=1e-6, epsilon=None, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

## (4) 모델 학습시키기

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Coding Time



## (5) 모델 평가하기

In [ ]:
loss_and_accuracy = model.evaluate(X_test, Y_test, batch_size=128)
print('loss : %.4f, accruracy : %.4f'%(loss_and_accuracy[0],loss_and_accuracy[1]))

## 1.5 BatchNormalization & DropOut

## (2) 모델링

<사용되는 Layer>

`Dropout` : 일부 뉴런을 drop하여 overfitting을 방지

https://keras.io/layers/core/#dropout
    
보통 batchnormalization, dropout은 동시에 사용하지 않음

In [ ]:
def CNN_Dropout(n_in, n_out):
    # Feature Extraction
    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3, 3), padding='same', input_shape=n_in))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Conv2D(32, (3, 3), padding='same',strides=(2, 2)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(ReLU())
    
    # Classifier
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dense(n_out, activation='softmax'))
    return model

def CNN_Dropout_func(n_in, n_out):
    input = Input(shape=(n_in))
    x = Conv2D(16, kernel_size=(3, 3), padding='same', input_shape=n_in)(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=n_in)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dense(n_out)(x)
    y = Activation('softmax')(x)
    model = Model(inputs = input, outputs = y)
    return model

model=CNN_Dropout_func(n_in, n_out)
model.summary()

from tensorflow.keras.utils import plot_model
%matplotlib inline
plot_model(model, show_shapes=True)

## (3) 모델의 학습과정 설정

In [ ]:
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=1e-6, epsilon=None, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

## (4) 모델 학습시키기

In [ ]:
earlystopper = EarlyStopping(monitor='val_accuracy', patience=7, verbose=0, mode='auto')
history = model.fit(X_train, Y_train, batch_size=128, epochs=30, validation_split=0.2, callbacks = [earlystopper])

## (5) 모델 평가하기

In [ ]:
loss_and_accuracy = model.evaluate(X_test, Y_test, batch_size=128)
print('loss : %.4f, accruracy : %.4f'%(loss_and_accuracy[0],loss_and_accuracy[1]))

## (6) 모델 저장하고 불러오기

저장하기 : model 객체의 내부 함수인 save() .h5 형식으로 저장할 수 있음 [1.모델의 구조, 2.학습된 파라미터, 3.complie() 설정]

In [ ]:
# Coding Time


불러오기 : load_model 함수로 .h5 파일에서 모델을 불러올 수 있음

In [ ]:
# Coding Time
